In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift
import json
from scipy import stats
import csv
import os.path
import tqdm

In [2]:
finalfile = "media results final final final.csv"

In [3]:
#create one aggregate file
import os
aggrfile = open("aggregate/{}".format(finalfile), "w+")
header = "n,p,eps,gam,gam_media,p_media,max_it,media_op,avg_ncluster,std_ncluster,avg_pwdist,std_pwdist,avg_niter,std_niter\n"
aggrfile.write(header)
aggrfile.close()
aggrfile = open("aggregate/{}".format(finalfile), "a+")
i = 0
for filename in os.listdir("aggregate/"):
    if filename.startswith('averages media'):
        infile = open("aggregate/{}".format(filename), "r")
        lines = infile.readlines()
        for line in lines:
            l = line.strip().split(',')
            if l[0] == "complete":
                l = l[1:]
            s = ','.join(l)
            print(s)
            s+='\n'
            aggrfile.write(s)
        infile.close()
        i+=1
aggrfile.close()
print("processed ", i, " files")

100,1.0,0.5,1.0,1.0,0.4,1000000,0.05;0.95,1.8573994796532654,0.13863021690404387,0.20608744565,0.37776031362146223,36.17,4.741936270761092
100,1.0,0.2,0.75,0.75,0.2,1000000,0.05;0.5;0.95,2.761920722507222,0.2792820927026703,0.17662319247999997,0.2821131395169076,63.64,7.851551992241323
100,1.0,0.3,0.75,0.75,0.2,1000000,0.05;0.5;0.95,1.5045068655127671,0.20430315427576048,0.07496370328999999,0.17317688289905125,69.52,29.028888432669337
100,1.0,0.4,0.5,0.5,0.2,1000000,0.5,1.0197607292384079,0.025682384956138078,0.004581205,0.04462959398976543,28.53,8.62396824453965
100,1.0,0.2,0.5,0.5,0.2,1000000,0.05;0.95,3.2081145016954458,0.9947527045567574,0.18881079122,0.3023647727805738,63.31,10.170957862383654
100,1.0,0.2,0.0,0.0,0.0,1000000,0.05;0.95,2.1217434562270636,0.3547635495737778,0.12142245311000001,0.21162822169836085,44.78,25.2376902687431
100,1.0,0.5,0.5,0.5,0.0,1000000,0.05;0.95,1.00020199959192,0.0020199959192001684,0.0008098395999999995,0.00542094661188745,10021.37,99997.84191607249

In [7]:
# results = pd.read_csv("aggregate/{}".format(filename))
# results.drop_duplicates(subset = ['graph', 'n', 'p', 'eps', 'gam', 'gr', 'pr', 'max_it'], inplace=True)
# results.to_csv("aggregate/{}".format(filename))
results = pd.read_csv("aggregate/{}".format(finalfile))
results.head()


,n,p,eps,gam,gam_media,p_media,max_it,media_op,avg_ncluster,std_ncluster,avg_pwdist,std_pwdist,avg_niter,std_niter
0,100,1.0,0.5,1.00,1.00,0.4,1000000,0.05;0.95,1.857399,0.138630,0.206087,0.377760,36.17,4.741936
1,100,1.0,0.2,0.75,0.75,0.2,1000000,0.05;0.5;0.95,2.761921,0.279282,0.176623,0.282113,63.64,7.851552
2,100,1.0,0.3,0.75,0.75,0.2,1000000,0.05;0.5;0.95,1.504507,0.204303,0.074964,0.173177,69.52,29.028888
3,100,1.0,0.4,0.50,0.50,0.2,1000000,0.5,1.019761,0.025682,0.004581,0.044630,28.53,8.623968
4,100,1.0,0.2,0.50,0.50,0.2,1000000,0.05;0.95,3.208115,0.994753,0.188811,0.302365,63.31,10.170958


In [8]:
# def nclusters_meanshift(data, bw):
#     ops = {i: data[i] for i in range(len(data))}
#     sorted_ops = sorted(ops.items(), key = lambda kv:(kv[1], kv[0]))
#     A=np.array([el[1] for el in sorted_ops]).reshape(-1,1)
#     clustering = MeanShift(bandwidth=bw).fit(A)
#     lbls = clustering.labels_
#     labels = np.arange(len(sorted_ops))
#     for i in range(len(labels)):
#         cl = lbls[i]
#         labels[sorted_ops[i][0]]=cl
#     cluster_participation_dict = {}
#     for l in labels:
#         if l not in cluster_participation_dict:
#             cluster_participation_dict[l] = 1
#         else:
#             cluster_participation_dict[l] += 1
#     #computing effective number of clusters using function explained in the paper
#     C_num = 0
#     C_den = 0
#     for k in cluster_participation_dict:
#         C_num += cluster_participation_dict[k]
#         C_den += ((cluster_participation_dict[k])**2)
#     C_num = (C_num**2)
#     C = C_num/C_den
#     return C

def nclusters(data, threshold):
    data = [float(el) for el in data]
    data = sorted(data)
    start = data[0]
    max_val = start + threshold
    c = (start, max_val)
    cluster = dict()
    for i in data:
        if i <= max_val:
            if c in cluster.keys():
                cluster[c] += 1
            else:
                cluster[c] = 1
        else:
            max_val = i + threshold
            c = (i, max_val)
            cluster[c] = 1
    #ora ho il dizionario con i cluster di una run
    C_num = len(data)**2
    C_den = 0
    for k in cluster.keys():
        C_den += cluster[k]*cluster[k]
    C = C_num / C_den
    return C

In [9]:
results.head()

,n,p,eps,gam,gam_media,p_media,max_it,media_op,avg_ncluster,std_ncluster,avg_pwdist,std_pwdist,avg_niter,std_niter
0,100,1.0,0.5,1.00,1.00,0.4,1000000,0.05;0.95,1.857399,0.138630,0.206087,0.377760,36.17,4.741936
1,100,1.0,0.2,0.75,0.75,0.2,1000000,0.05;0.5;0.95,2.761921,0.279282,0.176623,0.282113,63.64,7.851552
2,100,1.0,0.3,0.75,0.75,0.2,1000000,0.05;0.5;0.95,1.504507,0.204303,0.074964,0.173177,69.52,29.028888
3,100,1.0,0.4,0.50,0.50,0.2,1000000,0.5,1.019761,0.025682,0.004581,0.044630,28.53,8.623968
4,100,1.0,0.2,0.50,0.50,0.2,1000000,0.05;0.95,3.208115,0.994753,0.188811,0.302365,63.31,10.170958


In [30]:
import tqdm
# new_values_avg = []
# new_values_std = []
avg = []
std = []
nruns = []
for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    p_media = row['p_media']
    max_it = row['max_it']
    mo = ", ".join(row['media_op'].split(";"))
    name = f'media mo[{mo}] p{p_media} e{eps} g{gam} gm{gam} mi{max_it}'
    print(f'doing {name}')
    try:
        jsonfile = open(f'aggregate/final_opinions {name}.json')
        data = json.load(jsonfile)
        ncarray = []
        ncarray2 = []
        for nr in data.keys():
            finalops = list(data[nr])
            # nc = nclusters_meanshift(finalops, bw=0.01)
            nc2 = nclusters(finalops, 0.001)
            # ncarray.append(nc)
            ncarray2.append(nc2)
        # ncarray = np.array(ncarray)
        ncarray2 = np.array(ncarray2)
        # row["avg_ncluster_MS0.01"] = np.average(ncarray)
        # row["std_ncluster_MS0.01"] = np.std(ncarray)
        avg.append(np.average(ncarray2))
        std.append(np.std(ncarray2))
        nruns.append(nr)
    except FileNotFoundError:
        print(f"aggregate/final_opinions {name}.json file not found")
        # row["avg_ncluster_MS0.01"] = None
        # row["std_ncluster_MS0.01"] = None
        avg.append(None)
        std.append(None)
        nruns.append(None)
        continue

results["avg_ncluster_0.001"] = avg
results["std_ncluster_0.001"] = std
# results['nruns'] = nruns

doing media mo[0.05, 0.95] p0.4 e0.5 g1.0 gm1.0 mi1000000
doing media mo[0.05, 0.5, 0.95] p0.2 e0.2 g0.75 gm0.75 mi1000000
doing media mo[0.05, 0.5, 0.95] p0.2 e0.3 g0.75 gm0.75 mi1000000
doing media mo[0.5] p0.2 e0.4 g0.5 gm0.5 mi1000000
doing media mo[0.05, 0.95] p0.2 e0.2 g0.5 gm0.5 mi1000000
doing media mo[0.05, 0.95] p0.0 e0.2 g0.0 gm0.0 mi1000000
doing media mo[0.05, 0.95] p0.0 e0.5 g0.5 gm0.5 mi1000000
doing media mo[0.5] p0.1 e0.2 g0.0 gm0.0 mi1000000
doing media mo[0.05, 0.5, 0.95] p0.4 e0.3 g0.0 gm0.0 mi1000000
doing media mo[0.05, 0.95] p0.1 e0.3 g1.0 gm1.0 mi1000000
doing media mo[0.0] p0.0 e0.3 g0.75 gm0.75 mi1000000
doing media mo[0.5] p0.3 e0.3 g1.25 gm1.25 mi1000000
doing media mo[0.05, 0.95] p0.0 e0.2 g1.25 gm1.25 mi1000000
doing media mo[0.5] p0.4 e0.4 g0.0 gm0.0 mi1000000
doing media mo[0.05, 0.5, 0.95] p0.4 e0.3 g0.75 gm0.75 mi1000000
doing media mo[0.05, 0.95] p0.2 e0.3 g0.5 gm0.5 mi1000000
doing media mo[0.5] p0.5 e0.5 g1.25 gm1.25 mi1000000
doing media mo[0.05, 0

In [13]:
len(results)

577

In [31]:
results.to_csv(f"aggregate/{finalfile}")
results = pd.read_csv(f"aggregate/{finalfile}", index_col=[0])

In [32]:
results.head()

,n,p,eps,gam,gam_media,p_media,max_it,media_op,avg_ncluster,std_ncluster,...,avg_005_perc,10B_avg_entr,10B_std_entr,100B_avg_entr,100B_std_entr,100B_norm_entr,avg_ncluster_0.01,std_ncluster_0.01,avg_ncluster_0.001,std_ncluster_0.001
0,100,1.0,0.5,1.00,1.00,0.4,1000000,0.05;0.95,1.857399,0.138630,...,0.4737,0.650019,0.047919,0.650019,0.047919,0.162711,1.857399,0.137935,2.319842,0.460939
1,100,1.0,0.2,0.75,0.75,0.2,1000000,0.05;0.5;0.95,2.761921,0.279282,...,0.2838,1.130615,0.146573,1.132502,0.147791,0.283485,2.761921,0.277882,3.249417,0.554042
2,100,1.0,0.3,0.75,0.75,0.2,1000000,0.05;0.5;0.95,1.504507,0.204303,...,0.1251,0.552115,0.182304,0.552115,0.182304,0.138204,1.504507,0.203279,2.147410,0.549003
3,100,1.0,0.4,0.50,0.50,0.2,1000000,0.5,1.019761,0.025682,...,0.0053,0.095943,0.154546,0.096688,0.155011,0.024203,1.019761,0.025554,1.671637,0.380699
4,100,1.0,0.2,0.50,0.50,0.2,1000000,0.05;0.95,3.208115,0.994753,...,0.3266,1.175698,0.208922,1.414998,0.393034,0.354199,3.209878,0.996878,4.557575,1.649511


In [14]:
def entropy(opinions, n, nbins):
    bincounts, bin_edges = np.histogram(opinions, bins = np.linspace(0, 1, nbins))
    probabilities = bincounts/n
    entr = stats.entropy(probabilities)
    return entr

def nodeperc_withininterval(opinions, center, width):
    sortedops = sorted(opinions)
    sortedopsarr = np.array(sortedops)
    withinrange = np.where(np.logical_and(sortedopsarr>=(center-width), sortedopsarr<=(center+width)))
    withinrange = withinrange[0]
    perc = len(withinrange) / len(sortedops)
    return perc

def average_opinions(opinions):
    a = np.array(opinions)
    return np.average(a)

def std_opinions(opinions):
    a = np.array(opinions)
    return np.std(a)

def median_opinions(opinions):
    a = np.array(opinions)
    return np.median(a)

In [15]:
averageops = []
averagestdops = []
averagemedianops = []
avg_nodeperc_00 = []
avg_nodeperc_05 = []
avg_nodeperc_095 = []
avg_nodeperc_005 = []
for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    p_media = row['p_media']
    max_it = row['max_it']
    mo = ", ".join(row['media_op'].split(";"))
    name = f'media mo[{mo}] p{p_media} e{eps} g{gam} gm{gam} mi{max_it}'
    try:
        print(f'doing {name}')
        jsonfile = open(f'aggregate/final_opinions {name}.json')
        data = json.load(jsonfile)
        avgarray = []
        stdarray = []
        medarray = []
        arr00 = []
        arr05 = []
        arr095 = []
        arr005 = []
        for nr in data.keys():
            finalops = list(data[nr])
            avg = average_opinions(finalops)
            med = median_opinions(finalops)
            std = std_opinions(finalops)
            perc00 = nodeperc_withininterval(finalops, 0.0, 0.1)
            perc05 = nodeperc_withininterval(finalops, 0.5, 0.1)
            perc95 = nodeperc_withininterval(finalops, 0.95, 0.1)
            perc005 = nodeperc_withininterval(finalops, 0.05, 0.1)
            avgarray.append(avg)
            medarray.append(med)
            stdarray.append(std)
            arr00.append(perc00)
            arr05.append(perc05)
            arr095.append(perc95)
            arr005.append(perc005)
        avgarray = np.array(avgarray)
        medarray = np.array(medarray)
        stdarray = np.array(stdarray)
        arr00 = np.array(arr00)
        arr05 = np.array(arr05)
        arr095 = np.array(arr095)
        arr005 = np.array(arr005)
        averageops.append(np.average(avgarray))
        averagestdops.append(np.average(stdarray))
        averagemedianops.append(np.average(medarray))
        avg_nodeperc_00.append(np.average(arr00))
        avg_nodeperc_05.append(np.average(arr05))
        avg_nodeperc_095.append(np.average(arr095))
        avg_nodeperc_005.append(np.average(arr005))
    except FileNotFoundError:
        averageops.append(None)
        averagestdops.append(None)
        averagemedianops.append(None)
        avg_nodeperc_00.append(None)
        avg_nodeperc_05.append(None)
        avg_nodeperc_095.append(None)
        avg_nodeperc_005.append(None)

results["avg_opinion"] = averageops
results["std_opinions"] = averagestdops
results["median_opinion"] = averagemedianops
results["avg_00_perc"] = avg_nodeperc_00
results['avg_05_perc'] = avg_nodeperc_05
results['avg_095_perc'] = avg_nodeperc_095
results['avg_005_perc'] = avg_nodeperc_005

doing media mo[0.05, 0.95] p0.4 e0.5 g1.0 gm1.0 mi1000000
doing media mo[0.05, 0.5, 0.95] p0.2 e0.2 g0.75 gm0.75 mi1000000
doing media mo[0.05, 0.5, 0.95] p0.2 e0.3 g0.75 gm0.75 mi1000000
doing media mo[0.5] p0.2 e0.4 g0.5 gm0.5 mi1000000
doing media mo[0.05, 0.95] p0.2 e0.2 g0.5 gm0.5 mi1000000
doing media mo[0.05, 0.95] p0.0 e0.2 g0.0 gm0.0 mi1000000
doing media mo[0.05, 0.95] p0.0 e0.5 g0.5 gm0.5 mi1000000
doing media mo[0.5] p0.1 e0.2 g0.0 gm0.0 mi1000000
doing media mo[0.05, 0.5, 0.95] p0.4 e0.3 g0.0 gm0.0 mi1000000
doing media mo[0.05, 0.95] p0.1 e0.3 g1.0 gm1.0 mi1000000
doing media mo[0.0] p0.0 e0.3 g0.75 gm0.75 mi1000000
doing media mo[0.5] p0.3 e0.3 g1.25 gm1.25 mi1000000
doing media mo[0.05, 0.95] p0.0 e0.2 g1.25 gm1.25 mi1000000
doing media mo[0.5] p0.4 e0.4 g0.0 gm0.0 mi1000000
doing media mo[0.05, 0.5, 0.95] p0.4 e0.3 g0.75 gm0.75 mi1000000
doing media mo[0.05, 0.95] p0.2 e0.3 g0.5 gm0.5 mi1000000
doing media mo[0.5] p0.5 e0.5 g1.25 gm1.25 mi1000000
doing media mo[0.05, 0

In [16]:
results.head()

,n,p,eps,gam,gam_media,p_media,max_it,media_op,avg_ncluster,std_ncluster,...,avg_ncluster_0.00001,std_ncluster_0.00001,nruns,avg_opinion,std_opinions,median_opinion,avg_00_perc,avg_05_perc,avg_095_perc,avg_005_perc
0,100,1.0,0.5,1.00,1.00,0.4,1000000,0.05;0.95,1.857399,0.138630,...,7.993103,3.919469,97,0.523561,0.429741,0.594177,0.4737,0.0000,0.5263,0.4737
1,100,1.0,0.2,0.75,0.75,0.2,1000000,0.05;0.5;0.95,2.761921,0.279282,...,6.951949,2.552749,97,0.497376,0.331135,0.500063,0.2838,0.4315,0.2777,0.2838
2,100,1.0,0.3,0.75,0.75,0.2,1000000,0.05;0.5;0.95,1.504507,0.204303,...,4.577356,1.628646,97,0.484442,0.184689,0.495717,0.1251,0.7849,0.0900,0.1251
3,100,1.0,0.4,0.50,0.50,0.2,1000000,0.5,1.019761,0.025682,...,3.986371,1.406855,97,0.499679,0.030853,0.500073,0.0053,0.9904,0.0043,0.0053
4,100,1.0,0.2,0.50,0.50,0.2,1000000,0.05;0.95,3.208115,0.994753,...,12.360474,5.900746,97,0.494988,0.354297,0.525825,0.3266,0.2690,0.3021,0.3266


In [17]:
results.to_csv(f"aggregate/{finalfile}")
results = pd.read_csv(f"aggregate/{finalfile}", index_col=[0])

In [23]:
values_avg = []
values_std = []
values_avg_2 = []
values_std_2 = []
for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    p_media = row['p_media']
    max_it = row['max_it']
    mo = ", ".join(row['media_op'].split(";"))
    name = f'media mo[{mo}] p{p_media} e{eps} g{gam} gm{gam} mi{max_it}'
    print(f'doing {name}')
    try:
        jsonfile = open(f'aggregate/final_opinions {name}.json')
        data = json.load(jsonfile)
        entrarray = []
        entrarray2 = []
        for nr in data.keys():
            finalops = list(data[nr])
            entr = entropy(finalops, 100, 11)
            entr2 = entropy(finalops, 100, 101)
            entrarray.append(entr)
            entrarray2.append(entr2)
        entrarray = np.array(entrarray)
        entrarray2 = np.array(entrarray2)
        values_avg.append(np.average(entrarray))
        values_std.append(np.std(entrarray))
        values_avg_2.append(np.average(entrarray2))
        values_std_2.append(np.std(entrarray2))
    except FileNotFoundError:
        print("filenotfound")
        values_avg.append(None)
        values_std.append(None)
        values_avg_2.append(None)
        values_std_2.append(None)

results["10B_avg_entr"] = values_avg
results["10B_std_entr"] = values_std
results["100B_avg_entr"] = values_avg_2
results["100B_std_entr"] = values_std_2

doing media mo[0.05, 0.95] p0.4 e0.5 g1.0 gm1.0 mi1000000
doing media mo[0.05, 0.5, 0.95] p0.2 e0.2 g0.75 gm0.75 mi1000000
doing media mo[0.05, 0.5, 0.95] p0.2 e0.3 g0.75 gm0.75 mi1000000
doing media mo[0.5] p0.2 e0.4 g0.5 gm0.5 mi1000000
doing media mo[0.05, 0.95] p0.2 e0.2 g0.5 gm0.5 mi1000000
doing media mo[0.05, 0.95] p0.0 e0.2 g0.0 gm0.0 mi1000000
doing media mo[0.05, 0.95] p0.0 e0.5 g0.5 gm0.5 mi1000000
doing media mo[0.5] p0.1 e0.2 g0.0 gm0.0 mi1000000
doing media mo[0.05, 0.5, 0.95] p0.4 e0.3 g0.0 gm0.0 mi1000000
doing media mo[0.05, 0.95] p0.1 e0.3 g1.0 gm1.0 mi1000000
doing media mo[0.0] p0.0 e0.3 g0.75 gm0.75 mi1000000
doing media mo[0.5] p0.3 e0.3 g1.25 gm1.25 mi1000000
doing media mo[0.05, 0.95] p0.0 e0.2 g1.25 gm1.25 mi1000000
doing media mo[0.5] p0.4 e0.4 g0.0 gm0.0 mi1000000
doing media mo[0.05, 0.5, 0.95] p0.4 e0.3 g0.75 gm0.75 mi1000000
doing media mo[0.05, 0.95] p0.2 e0.3 g0.5 gm0.5 mi1000000
doing media mo[0.5] p0.5 e0.5 g1.25 gm1.25 mi1000000
doing media mo[0.05, 0

In [24]:
results.to_csv(f"aggregate/{finalfile}")
results = pd.read_csv(f"aggregate/{finalfile}", index_col=[0])
results.head()

,n,p,eps,gam,gam_media,p_media,max_it,media_op,avg_ncluster,std_ncluster,...,std_opinions,median_opinion,avg_00_perc,avg_05_perc,avg_095_perc,avg_005_perc,10B_avg_entr,10B_std_entr,100B_avg_entr,100B_std_entr
0,100,1.0,0.5,1.00,1.00,0.4,1000000,0.05;0.95,1.857399,0.138630,...,0.429741,0.594177,0.4737,0.0000,0.5263,0.4737,0.650019,0.047919,0.650019,0.047919
1,100,1.0,0.2,0.75,0.75,0.2,1000000,0.05;0.5;0.95,2.761921,0.279282,...,0.331135,0.500063,0.2838,0.4315,0.2777,0.2838,1.130615,0.146573,1.132502,0.147791
2,100,1.0,0.3,0.75,0.75,0.2,1000000,0.05;0.5;0.95,1.504507,0.204303,...,0.184689,0.495717,0.1251,0.7849,0.0900,0.1251,0.552115,0.182304,0.552115,0.182304
3,100,1.0,0.4,0.50,0.50,0.2,1000000,0.5,1.019761,0.025682,...,0.030853,0.500073,0.0053,0.9904,0.0043,0.0053,0.095943,0.154546,0.096688,0.155011
4,100,1.0,0.2,0.50,0.50,0.2,1000000,0.05;0.95,3.208115,0.994753,...,0.354297,0.525825,0.3266,0.2690,0.3021,0.3266,1.175698,0.208922,1.414998,0.393034


In [25]:
min(results['100B_avg_entr'])

0.0

In [26]:
def rescale(x, n, bins):
    minval = 0.0
    return x / maxval

# i = 0
# norm_entr = []
# for index,row in results.iterrows():
#     print(f"doing {i}th row")
#     entr = row['10B_avg_entr']
#     normentr = rescale(entr, 100, 11)
#     norm_entr.append(norm_entr)
#     i+=1
# results['10B_norm_entr'] = norm_entr
# results.head()
maxval = entropy(np.random.uniform(0.0, 1.0, size=100), 100, 101)

results['100B_norm_entr'] = results['100B_avg_entr'].apply(lambda x : x/maxval )

In [27]:
results.columns

Index(['n', 'p', 'eps', 'gam', 'gam_media', 'p_media', 'max_it', 'media_op',
       'avg_ncluster', 'std_ncluster', 'avg_pwdist', 'std_pwdist', 'avg_niter',
       'std_niter', 'avg_ncluster_0.00001', 'std_ncluster_0.00001', 'nruns',
       'avg_opinion', 'std_opinions', 'median_opinion', 'avg_00_perc',
       'avg_05_perc', 'avg_095_perc', 'avg_005_perc', '10B_avg_entr',
       '10B_std_entr', '100B_avg_entr', '100B_std_entr', '100B_norm_entr'],
      dtype='object')

In [28]:
dfnc = results[['100B_avg_entr', '10B_avg_entr', '10B_norm_entr', '100B_norm_entr', 'avg_ncluster', 'avg_ncluster_0.1', 'avg_ncluster_0.01', 'avg_ncluster_0.00001']]

KeyError: "['10B_norm_entr', 'avg_ncluster_0.1', 'avg_ncluster_0.01'] not in index"

In [27]:
dfnc.corr()

,100B_avg_entr,10B_avg_entr,10B_norm_entr,100B_norm_entr,avg_ncluster,avg_ncluster_0.1,avg_ncluster_0.01,avg_ncluster_0.00001
100B_avg_entr,1.000000,0.913800,0.913800,1.000000,0.820608,0.875910,0.838463,0.366699
10B_avg_entr,0.913800,1.000000,1.000000,0.913800,0.933147,0.973849,0.643657,0.093953
10B_norm_entr,0.913800,1.000000,1.000000,0.913800,0.933147,0.973849,0.643657,0.093953
100B_norm_entr,1.000000,0.913800,0.913800,1.000000,0.820608,0.875910,0.838463,0.366699
avg_ncluster,0.820608,0.933147,0.933147,0.820608,1.000000,0.967021,0.501182,0.073397
avg_ncluster_0.1,0.875910,0.973849,0.973849,0.875910,0.967021,1.000000,0.627280,0.117145
avg_ncluster_0.01,0.838463,0.643657,0.643657,0.838463,0.501182,0.627280,1.000000,0.554842
avg_ncluster_0.00001,0.366699,0.093953,0.093953,0.366699,0.073397,0.117145,0.554842,1.000000
